# Lecture 3

# The story so far

* Transform a system of polynomial equations $f_i(x) = 0$ to a *lower diagonal* one, where $f_i$ only depends on $x_1, \ldots, x_i$.
* Solve $f_1(x_1) = 0$, substitute and solve $f_2(x_1^*, x_2)$, etc.

Today, we focus on the canonical task of scalar root-finding, and the numerical issues that may arise when using it to solve the system of equations by substitution.

# Contents

* Scalar root-finding
* Finding multiple roots
* Computing with finite accuracy
* The (in)stability of Gaussian elimination


# Scalar root-finding

> Assuming (for the moment) that $f : \mathbb{R} \rightarrow \mathbb{R}$ has at least one real root, how do we find it?

* bisection
* Fixed-point iteration
* Newton's method

## Bi-section

* Start with $x_0, x_1$ such that $f(x_0)f(x_1) < 0$
* pick $x_2 = (x_0 + x_1) / 2$ and check on which interval the sign changes
* repeat

In [24]:
# copied from: https://personal.math.ubc.ca/~pwalls/math-python/roots-optimization/bisection/

def bisection(f,a,b,N):
    '''Approximate solution of f(x)=0 on interval [a,b] by bisection method.

    Parameters
    ----------
    f : function
        The function for which we are trying to approximate a solution f(x)=0.
    a,b : numbers
        The interval in which to search for a solution. The function returns
        None if f(a)*f(b) >= 0 since a solution is not guaranteed.
    N : (positive) integer
        The number of iterations to implement.

    Returns
    -------
    x_N : number
        The midpoint of the Nth interval computed by the bisection method. The
        initial interval [a_0,b_0] is given by [a,b]. If f(m_n) == 0 for some
        midpoint m_n = (a_n + b_n)/2, then the function returns this solution.
        If all signs of values f(a_n), f(b_n) and f(m_n) are the same at any
        iteration, the bisection method fails and return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> bisection(f,1,2,25)
    1.618033990263939
    >>> f = lambda x: (2*x - 1)*(x - 3)
    >>> bisection(f,0,1,10)
    0.5
    '''
    if f(a)*f(b) >= 0:
        print("Bisection method fails.")
        return None
    a_n = a
    b_n = b
    for n in range(1,N+1):
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n
            b_n = m_n
        elif f(b_n)*f_m_n < 0:
            a_n = m_n
            b_n = b_n
        elif f_m_n == 0:
            print("Found exact solution.")
            return m_n
        else:
            print("Bisection method fails.")
            return None
    return (a_n + b_n)/2

In [25]:
f = lambda x: x*(x-1)

root = bisection(f,.5,2,100)    

print('root  = ', root)
print('error = ', abs(root - 1))

Found exact solution.
root  =  1.0
error =  0.0


* Method only works when $f$ changes sign at the root
* Method converges lineary

## Fixed-point iteration

* Express $f(x) = 0$ as $g(x) = x$
* Apply *fixed-point iteration* $x_{k+1} = g(x_k)$.

In [30]:
g = lambda x : x**2
x = 2

for k in range(10):
    x = g(x)
    print('error = ', abs(x - 1))

error =  3
error =  15
error =  255
error =  65535
error =  4294967295
error =  18446744073709551615
error =  340282366920938463463374607431768211455
error =  115792089237316195423570985008687907853269984665640564039457584007913129639935
error =  13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006084095
error =  179769313486231590772930519078902473361797697894230657273430081157732675805500963132708477322407536021120113879871393357658789768814416622492847430639474124377767893424865485276302219601246094119453082952085005768838150682342462881473913110540827237163350510684586298239947245938479716304835356329624224137215


* Requires clever choice of $g$
* Some choices are better than others
* Method converges *at least* lineary, sometimes faster

## Newton's method

Construct $g$ such that $g'(x^*) = 0$:

$$g(x) = x - f(x)/f'(x).$$

* What happens when $f'(x^*) = 0$?

In [33]:
def newton(f,Df,x0,max_iter,epsilon=1e-16):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> Df = lambda x: 2*x - 1
    >>> newton(f,Df,1,1e-8,10)
    Found solution after 5 iterations.
    1.618033988749989
    '''
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None

In [37]:
f  = lambda x: x*(x-1)
df = lambda x: 2*x-1

root = newton(f,df,2,100)    

print('root  = ', root)
print('error = ', abs(root - 1))

Found solution after 6 iterations.
root  =  1.0
error =  0.0


* Quadratic convergence if $f'(x_*)\not = 0$, linear convergence otherwise
* Need to start sufficiently close to a root

## Basin of attraction

## Complex roots

# Finding multiple roots

> Supposedly, there will be multipe roots, how do we find them all?

## Horner's method

## The companion matrix

# Computing in finite accuracy

> We cannot compute in $\mathbb{R}$, so what do we do?

## Floating point arithemetic

## Error analysis

# The (in)stability of Gaussian elimination

> How do round-off errors propagate through Gaussian elimination?

## An example

## Gaussian elimination is unstable!